# IMPORT LIBRARY

In [ ]:
import os
import time
import pandas as pd
from datetime import datetime,timedelta
start = datetime.now()
from google.colab import drive
drive.mount('/content/drive')

!pip install xlsxwriter
import xlsxwriter
!pip install gspread==5.7.2
import gspread
from google.oauth2.service_account import Credentials

creds = Credentials.from_service_account_file('your-credentials.json')
gc = gspread.authorize(creds)



# Prepare Data
Displays pull data's date and upload file metabase data FAS

In [ ]:
yesterday = datetime.today()-timedelta(days=1)+timedelta(hours=7)
tanggal = ' '+yesterday.strftime('%d %B %Y')
print('Tanggal kemarin:',tanggal)

print()
print()
metabase = pd.read_csv('fas.csv',encoding='ISO-8859-1')
df=metabase.copy()
df

# Cleaning Data Region

In [ ]:
#Grouping Region
intimi = ['Bali','Nusa','Maluku','Papua']
northern = ['Sumatera 1','Sumatera 3','Sumatera 4']
southern = ['Sumatera 2','Sumatera 5','Sumatera 6']

#Replace
df.loc[df['dest_hub_region'].isin(intimi),'dest_hub_region'] = 'Intim'
df.loc[df['dest_hub_region'].isin(northern),'dest_hub_region'] = 'Northern Sumatera'
df.loc[df['dest_hub_region'].isin(southern),'dest_hub_region'] = 'Southern Sumatera'
df['dest_hub_region'].unique()

# Export Excel per Region To Drive

In [ ]:
################ EXPORT EXCEL #####################################


for i in df['dest_hub_region'].unique():
    pd.options.mode.chained_assignment = None
    filenames  = 'RawData FAS '+i+ ' '+ tanggal +'.xlsx'
    df_i = df[df['dest_hub_region']==i]

    with pd.ExcelWriter('/content/drive/MyDrive/FAS/'+ filenames) as writer:
        # raw data
        df_i.to_excel(writer, sheet_name= 'Raw Data',startrow= 1, index=False, header=False)
        workbook_raw  = writer.book
        worksheet = writer.sheets['Raw Data']
        header_format_raw = workbook_raw.add_format({'bold': True,'fg_color': 'yellow','border': 1,'align':'center','valign':'vcenter'})
          # biar header di tiap tabel berwarna
        worksheet_raw = writer.sheets['Raw Data']
        for col_num, value in enumerate(df_i.columns.values):
          #write to second row
          worksheet_raw.write(0, col_num, value, header_format_raw)
          column_len = df_i[value].astype(str).str.len().max()
          column_len = max(column_len, len(value))
          worksheet_raw.set_column(col_num, col_num, column_len)

        print(i, 'DONE to excel')

# Pivotable FAS
menampilkan kolom FAS dimana jika first valid delivery = delivery_success, maka flag Hit, sedangkan jika tidak sama maka flag Miss dan eksekusi pivot

In [ ]:
df_pivot=df.copy()
df_pivot.loc[df['first_valid_delivery_attempt_datetime'] == df['delivery_success_datetime'],'FAS?'] = "1"
df_pivot.loc[df['first_valid_delivery_attempt_datetime'] != df['delivery_success_datetime'],'FAS?'] = "0"


pivot_data = pd.pivot_table(df_pivot, values='tracking_id', index=['dest_hub_name','dest_hub_region'], columns='FAS?', aggfunc='count')
pivot_data.columns.name = None
pivot_data.reset_index(inplace=True)
pivot_data.fillna(0, inplace=True)
pivot_data['Grand_Total'] = pivot_data.sum(axis=1)
pivot_data.rename(columns = {'dest_hub_name':'Dest Hub Name','dest_hub_region':'Region Grup',
                             '0':'Miss','1':'Hit','Grand_Total':'Grand Total'}, inplace = True)
pivot_data.value_counts()

# Update Data & Process Excel

In [ ]:
print('>> Updating Sheet <<')

ws = gc.open_by_url('your_gsheet_url').worksheet('Raw FAS')

data_metabase_new = pivot_data.reindex(columns = pivot_data.columns)
ws.batch_clear(["A:E"])
ws.update('A1',[data_metabase_new.columns.values.tolist()] + data_metabase_new.values.tolist(), raw=False)

print('DONE to Sheet')
print()


print('>> Ekstracting Excel <<')

with pd.ExcelWriter('/content/drive/MyDrive/FAS/ALL RAWDATA FAS ' +tanggal+'.xlsx') as writer:
        # raw data
        df_pivot.to_excel(writer, sheet_name= 'Raw Data',startrow= 1, index=False, header=False)
        workbook_raw  = writer.book
        worksheet = writer.sheets['Raw Data']
        header_format_raw = workbook_raw.add_format({'bold': True,'fg_color': 'yellow','border': 1,'align':'center','valign':'vcenter'})
          # biar header di tiap tabel berwarna
        worksheet_raw = writer.sheets['Raw Data']
        for col_num, value in enumerate(df_pivot.columns.values):
          #write to second row
          worksheet_raw.write(0, col_num, value, header_format_raw)
          column_len = df_pivot[value].astype(str).str.len().max()
          column_len = max(column_len, len(value))
          worksheet_raw.set_column(col_num, col_num, column_len)

        pivot_data.to_excel(writer, sheet_name= 'Pivot',startrow= 1, index=False, header=False)
        workbook_raw  = writer.book
        worksheet = writer.sheets['Pivot']
        header_format_raw = workbook_raw.add_format({'bold': True,'fg_color': 'red','border': 1,'align':'center','valign':'vcenter'})
          # biar header di tiap tabel berwarna
        worksheet_raw = writer.sheets['Pivot']
        for col_num, value in enumerate(pivot_data.columns.values):
          #write to second row
          worksheet_raw.write(0, col_num, value, header_format_raw)
          column_len = pivot_data[value].astype(str).str.len().max()
          # column_len = max(column_len, len(value))
          worksheet_raw.set_column(col_num, col_num, column_len)
print('DONE to excel')

In [ ]:
# DONE

#selesai
end = datetime.now()
code_runtime = (end - start).total_seconds()/60
print("elapsed_time = ",code_runtime,'minutes')